In [3]:
import os, sys
import numpy as np
from constants import GAS_CONST

In [4]:
def set_paths():
    """
    Set the absolute path to required files on the current machine.

    Returns
    -------
    reactionlist_path     : str
                            path to the file `complete_reactionlist.dat`
    rateconstantlist_path : str
                            path to the file `complete_rateconstantlist.dat`
    compositionlist_path  : str
                            path to the file `compositionlist.dat`
    """
    module_dir = os.getcwd().split('ligpy_utils')[0]
    reactionlist_path = module_dir + '/data/complete_reaction_list.dat'
    rateconstantlist_path = module_dir + '/data/complete_rateconstant_list.dat'
    compositionlist_path = module_dir + '/data/compositionlist.dat'
    return reactionlist_path, rateconstantlist_path, compositionlist_path

In [5]:
reactionlist, rateconstantlist, compositionlist = set_paths()

In [6]:
print(reactionlist)

/Users/chowdhury/Documents/kmpy/kmpy/data/complete_reaction_list.dat


In [7]:
class Reaction(object):
    
    
    def __init__(self):
        self.reactants_names = []
        self.products_names = []
        self.uniqueSpeciesList = []
        #species_names = []
        
   
    def getReactantsName(self, line):
        
        for spec in line.split(','):
            if float(spec.split('_')[0].split()[0]) < 0:
                self.reactants_names.append((spec.split('_')[0].split()[0],
                                          spec.split('_')[1].split()[0]))
            #print(self.species_names)
        return self.reactants_names
    
    def getProductsName(self, line):
        
        for spec in line.split(','):
            if float(spec.split('_')[0].split()[0]) > 0:
                self.products_names.append((spec.split('_')[0].split()[0],
                                          spec.split('_')[1].split()[0]))
            #print(self.species_names)
        return self.products_names
    
    def uniqueSpeciesName(self, line, species_list):
        #self.uniqueSpeciesList = species_list
        for spec in line.split(','):
            #self.uniqueSpeciesList = species_list
            # If the species has already been added to the list then move on.
            if spec.split('_')[1].split()[0] in species_list:
                self.uniqueSpeciesList = species_list
                continue
            else:
                #print(self.uniqueSpeciesList)
                self.uniqueSpeciesList = species_list
                self.uniqueSpeciesList.append(spec.split('_')[1].split()[0])
            #print(spec.split('_')[1].split()[0])
        return self.uniqueSpeciesList
    

In [64]:
def build_species_list(reaction_file):
    """
    Build reactnat and product list for each reaction. Also builds a list
    of unique species in the mechanism
    Parameters
    ----------
    reaction_file       : str
                           path to the file `complete_reaction_list.dat`
    Returns
    __________

    reactant_list       : list
                         a list of the reactants and their stoichiometric
                         coeffs for each reaction
    product_list        : list
                         a list of the products and their stoichiometric
                         coeffs for each reaction
    species_list        : list
                        a list of unique species in the mechanism
    """

    #initializing reactant, product and unique species list
    reactant_list = []
    product_list = []
    species_name = []

    for line in open(reaction_file, 'r').readlines():
        reac = Reaction()
        reactant_list.append(reac.getReactantsName(line))
        product_list.append(reac.getProductsName(line))
        current_species = species_name
        #print(current_species)
        species_list = reac.uniqueSpeciesName(line, current_species)
        #print(species_name)
    species_list.sort()

    return reactant_list, product_list, species_list

reactants_list, products_list, unique_species = build_species_list(reactionlist)
    

In [65]:
print(unique_species)

['A', 'B', 'C', 'D', 'E', 'F']


In [127]:
print(products_list)


[[('1', 'C'), ('1', 'D')], [('1', 'E')], [('1', 'D'), ('1', 'E')], [('1', 'D'), ('1', 'F')]]


In [11]:
reac_prod_list = [react + prod for react, prod in zip(reactants_list, products_list)]
# for react, prod in zip(reactants, products):
#     total.append(react + prod)
#print(total)

In [12]:
reac_prod_list

[[('-1', 'A'), ('-2', 'B'), ('1', 'C'), ('1', 'D')],
 [('-1', 'A'), ('1', 'E')],
 [('-1', 'B'), ('-1', 'C'), ('1', 'D'), ('1', 'E')],
 [('-1', 'A'), ('1', 'D'), ('1', 'F')]]

In [13]:
#generating a dictionary of unique species from the species_list
speciesindices = {species_list[i]:i for i in range(0, len(species_list))}

indices_to_species = dict(zip(speciesindices.values(), speciesindices.keys()))

In [14]:
#speciesindices

In [15]:
class Kinetic_params(object):
      
    def __init__(self):
        self.forward_rate_params = []
        self.forward_rates = []
        #self.forward_E = []
        #self.uniqueSpeciesList = []
        #species_names = []
    
    def getForwardRateParameters(self, line):
        
        self.forward_rate_params = [line.split(' ')[0], line.split(' ')[1],
                      line.split(' ')[2].split()[0]]
      
        return self.forward_rate_params
    
    def getForwardRateConstant(self, parameters, T):
        
        self.forward_rates = eval(params[0]) * np.exp(- eval(params[2])/
                                                               (GAS_CONST * temp))
        return self.forward_rates

In [16]:
forward_rate_constants = []
temp = 298

In [17]:
for line in open(rateconstantlist, 'r').readlines():
    f_params = Kinetic_params()
    params = f_params.getForwardRateParameters(line)
    forward_rate_constants.append(f_params.getForwardRateConstant(params, temp))

In [18]:
forward_rate_constants

[2.4162838852855017e-16,
 2.4162838852855017e-16,
 2.4162838852855017e-16,
 2.4162838852855017e-16]

In [19]:
def build_reac_prod_dict(reac_list, prod_list, speciesindices):
    """
    Build a dictionary of the reactants involved in each reaction,
    along with their stoichiometric coefficients.  The keys of the
    dictionary are the reaction numbers, the values are lists of lists
    [[reactant1index, -1*coeff1],...]
    Parameters
    ----------
    completereactionlist : str
                           path to the file `complete_reaction_list.dat`
    speciesindices       : dict
                           the dictionary speciesindices from
                           get_speciesindices()
    Returns
    -------
    reactant_dict : dict
                    a dictionary where keys are reaction numbers and values
                    are lists of lists with the reactants and their
                    stoichiometric coefficients for each reaction
    """
    reactant_dict = {}
    for rxnindex, reaction in enumerate(reac_list):
        reactants = []
        #
        for x in range(len(reaction)):
            # if the species is a reactant
         #   if float(x.split('_')[0]) < 0:
            reactants.append([speciesindices[reaction[x][1]],
                                -1*float(reaction[x][0])])
            #    in preceding line: *-1 because I want the |stoich coeff|
        reactant_dict[rxnindex] = reactants
        
    products_dict = {}
    for rxnindex, reaction in enumerate(prod_list):
        products = []
        #
        for x in range(len(reaction)):
            # if the species is a reactant
         #   if float(x.split('_')[0]) < 0:
            products.append([speciesindices[reaction[x][1]],
                                1*float(reaction[x][0])])
            #    in preceding line: *-1 because I want the |stoich coeff|
        products_dict[rxnindex] = products
    return reactant_dict, products_dict

In [135]:
reac_dict, prod_dict = build_reac_prod_dict(reactants_list, products_list, speciesindices)
reac_dict


{0: [[0, 1.0], [1, 2.0]],
 1: [[0, 1.0]],
 2: [[1, 1.0], [2, 1.0]],
 3: [[0, 1.0]]}

In [21]:
def build_reac_species_dict(reacprodlist, specieslist):
    """
    Build a dictionary where keys are species and values are lists with the
    reactions that species is involved in, that reaction's sign in the net
    rate equation, and the stoichiometric coefficient of the species in that
    reaction.
    Parameters
    ----------
    reacprodlist : list
                        a list of both reactants and products and their 
                        stoichiometric co-effs
    specieslist  : list
                        a list of unique species in the mecahnism
    
    Returns
    -------
    reac_species : dict
                   keys are the species in the model; values are lists of
                   [reaction that species is involved in,
                   sign of that species in the net rate equation,
                   stoichiometric coefficient]
    """
    #specieslist = get_specieslist(set_paths()[0])
    reac_species = {}
    for species in species_list:
        # This loop makes a list of which reactions "species" takes part in
        # and what sign that term in the net rate eqn has
        # and what the stoichiometric coefficient is
    
        reactions_involved = []
        for rxnindex, reac_list in enumerate (reacprodlist):
            for x in range(len(reac_list)):
                # If the species being iterated over is part of this reaction
                if species == reac_list[x][1]:
                    # if the species is a reactant
                    if float(reac_list[x][0]) < 0:
                        reactions_involved.append(
                            [rxnindex, -1, str(-1), '+'+str(1)])
                    
                    # if the species is a product
                    if float(reac_list[x][0]) > 0:
                        reactions_involved.append(
                            [rxnindex, 1, '+'+str(1), str(-1)])
    
        reac_species[species] = reactions_involved
    return reac_species

In [136]:
reac_species = build_reac_species_dict(reac_prod_list, species_list)
reac_species['B']

[[0, -1, '-1', '+1'], [2, -1, '-1', '+1']]

In [56]:
def build_rate_eqn(k_mat, r_dict, s_indices, human, forward):

    """ This function writes the list of rate expressions for each reaction.
    Parameters
    ----------
    kmat               : list
                         A list of reaction rate contstants (k_forward or k_reverse)
    r_dict             : dictionary
                         reactant or product directory
    s_indices          : dict
                         the reverse of speciesindices (keys are the indices
                         and values are the species)
    human              : str, optional
                         indicate whether the output of this function should
                         be formatted for a human to read ('yes'). Default
                         is 'no'
    forward             : str
                        reaction type,if 'yes', it is forward reaction
                        default is 'yes'
    Returns
    -------
    rates_list : list
                A list of the rate expressions for all the reactions in the mecahnism
    """

    rates_list = []
    for i, line in enumerate(k_mat):
        if forward == 'yes':
            rate = 'rate_f[%s] = kf(T,%s) ' % (i, i)
        else:
            rate = 'rate_r[%s] = kr(T,%s) ' % (i, i)
        concentrations = ''
        for entry in r_dict[i]:
            if entry == 'n':   # if there is no reaction
                concentrations = '* 0'
                break
            else:
                if human == 'no':
                    concentrations += '* y[%s]**%s ' % (entry[0], entry[1])
                elif human == 'yes':
                    concentrations += '* [%s]**%s ' % \
                        (s_indices[entry[0]], entry[1])
                else:
                    raise ValueError('human must be a string: yes or no')
        rate += concentrations

        #rate = rate_reactant + rate_product
        rates_list.append(rate)
        
    return rates_list


rates_f = build_rate_eqn(forward_rate_constants, reac_dict, indices_to_species, human = 'no', forward = 'yes')
rates_r = build_rate_eqn(forward_rate_constants, prod_dict, indices_to_species, human = 'no', forward = 'yes')


print(rates_f)

['rate_f[0] = kf(T,0) * y[0]**1.0 * y[1]**2.0 ', 'rate_f[1] = kf(T,1) * y[0]**1.0 ', 'rate_f[2] = kf(T,2) * y[1]**1.0 * y[2]**1.0 ', 'rate_f[3] = kf(T,3) * y[0]**1.0 ']


In [57]:
def build_dydt_list(rates_forward, rates_reverse, specieslist, species_rxns, human='no'):
    """This function returns the list of dydt expressions generated for all
    the reactions from rates_list.
    Parameters
    ----------
    rates_list         : list
                         the output of build_rates_list()
    specieslist        : list
                         a list of all the species in the kinetic scheme
    species_rxns       : dict
                         dictionary where keys that are the model species and
                         values are the reactions they are involved in
    human              : str, optional
                         indicate whether the output of this function should
                         be formatted for a human to read ('yes'). Default
                         is 'no'
    Returns
    -------
    dydt_expressions : list
                       expressions for the ODEs expressing the concentration
                       of each species with time
    """
    dydt_expressions = []
    for species in specieslist:
        rate_formation = 'd[%s]/dt = ' % (species)
        # "entry" is [reaction#, sign of that reaction, coefficient]
        for entry in species_rxns[species]:
            if human == 'no':
                rate_formation += '%s*%s' % \
                    (entry[2], rates_forward[entry[0]].split(' = ')[1])
                rate_formation += '%s*%s' % \
                    (entry[3], rates_reverse[entry[0]].split(' = ')[1])
            elif human == 'yes':
                rate_formation += '%s*rate_f[%s] ' % (entry[2], entry[0])
                rate_formation += '%s*rate_r[%s] ' % (entry[3], entry[0])
            else:
                raise ValueError('human must be a string: yes or no')
        dydt_expressions.append(rate_formation)
    return dydt_expressions


In [58]:
dydt_list = build_dydt_list(rates_f, rates_r, species_list, reac_species, human='no')

In [59]:
dydt_list

['d[A]/dt = -1*kf(T,0) * y[0]**1.0 * y[1]**2.0 +1*kf(T,0) * y[2]**1.0 * y[3]**1.0 -1*kf(T,1) * y[0]**1.0 +1*kf(T,1) * y[4]**1.0 -1*kf(T,3) * y[0]**1.0 +1*kf(T,3) * y[3]**1.0 * y[5]**1.0 ',
 'd[B]/dt = -1*kf(T,0) * y[0]**1.0 * y[1]**2.0 +1*kf(T,0) * y[2]**1.0 * y[3]**1.0 -1*kf(T,2) * y[1]**1.0 * y[2]**1.0 +1*kf(T,2) * y[3]**1.0 * y[4]**1.0 ',
 'd[C]/dt = +1*kf(T,0) * y[0]**1.0 * y[1]**2.0 -1*kf(T,0) * y[2]**1.0 * y[3]**1.0 -1*kf(T,2) * y[1]**1.0 * y[2]**1.0 +1*kf(T,2) * y[3]**1.0 * y[4]**1.0 ',
 'd[D]/dt = +1*kf(T,0) * y[0]**1.0 * y[1]**2.0 -1*kf(T,0) * y[2]**1.0 * y[3]**1.0 +1*kf(T,2) * y[1]**1.0 * y[2]**1.0 -1*kf(T,2) * y[3]**1.0 * y[4]**1.0 +1*kf(T,3) * y[0]**1.0 -1*kf(T,3) * y[3]**1.0 * y[5]**1.0 ',
 'd[E]/dt = +1*kf(T,1) * y[0]**1.0 -1*kf(T,1) * y[4]**1.0 +1*kf(T,2) * y[1]**1.0 * y[2]**1.0 -1*kf(T,2) * y[3]**1.0 * y[4]**1.0 ',
 'd[F]/dt = +1*kf(T,3) * y[0]**1.0 -1*kf(T,3) * y[3]**1.0 * y[5]**1.0 ']

In [61]:
paths = set_paths()

In [110]:
specieslist = build_species_list(paths[0])
output_dict = {specieslist[2][i]:i for i in range(0, len(specieslist[2]))}

In [124]:
output_dict_reverse = dict(zip(output_dict.values(), output_dict.keys()))
print(output_dict_reverse[1])
print(len(specieslist[2]))

B
6


In [126]:
for i in range(len(specieslist[2])):
    spec = output_dict_reverse[i]
    print(output_dict[spec])
    

0
1
2
3
4
5


test_correct_format (__main__.TestGetSpecieslist)
Are the entries in specieslist strings longer than 1 character? ... /Users/chowdhury/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:27: ResourceWarning: unclosed file <_io.TextIOWrapper name='/Users/chowdhury/Documents/kmpy/kmpy/data/complete_reaction_list.dat' mode='r' encoding='UTF-8'>
FAIL
test_add (__main__.TestNotebook) ... ERROR

ERROR: test_add (__main__.TestNotebook)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-77-2b366691654a>", line 6, in test_add
    self.assertEqual(add(2, 2), 4)
NameError: name 'add' is not defined

FAIL: test_correct_format (__main__.TestGetSpecieslist)
Are the entries in specieslist strings longer than 1 character?
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-72-02d228ecfcea>", line 17, in test_correct_format
    self.assertGre